In [12]:
import cv2
import numpy as np

In [13]:
img_dir = '/Users/crossrunway/vsCODE/RobotProject/backend/uploads/911gt3rs.png'

In [14]:
# Read in image
img = cv2.imread(img_dir, 0)
img = 255 - img
img = cv2.resize(img, (6000,6000))
h,w = img.shape
cutter_size = 20
buffer = int(cutter_size * 1.25)
img2 = np.zeros((h+buffer*2, w+buffer*2), np.uint8)
img2[buffer:buffer+h, buffer:buffer+w] = img
img = img2.copy()

h,w = img.shape

In [15]:
# Define parameters
pixels_to_inch = 1000
depth_per_cut = 125 # Value in thousands
depth = 500 # Value in thousands
jog_speed = "15"
feed_speed = "9"
plunge_speed = "3"
current_depth = 0
cuts = 2

In [16]:
# Find the center of the part - there should only be one contour
contours, _ = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
contour = contours[1]

kernel = np.ones((20,20), np.uint8)
dilate = cv2.dilate(img, kernel, iterations = 4)

contours_dilate = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contour_dilate = contours_dilate[0]

In [17]:
# Change the contours from arrays to lists
outter = []
for point in contour_dilate[0]:
    outter.append(tuple(point[0]))
inner = []
for point in contour:
    inner.append(tuple(point[0]))

h,w = img.shape
bg = np.zeros((h,w,3), np.uint8)
bg = img.copy()
bg = cv2.drawContours(bg, [contour], -1, (255,255,0), 2)
bg = cv2.drawContours(bg, contour_dilate, -1, (0,0,255), 2)
# cv2.imshow('img', cv2.resize(bg, (987,987)))
# cv2.waitKey(0)
# cv2.destroyAllWindows()

bg = np.zeros((h,w,3), np.uint8)
for i in range(len(inner)-1):
    a = inner[i]
    b = inner[i-1]
    cv2.line(bg, a, b, (0,255,0), cutter_size)
for i in range(len(outter)-1):
    a = outter[i]
    b = outter[i+1]
    cv2.line(bg, a, b, (255,0,255), cutter_size)
    
bg = cv2.drawContours(bg, [contour], -1, (255,255,0), 2)
bg = cv2.drawContours(bg, contour_dilate, -1, (0,0,255), 2)
# cv2.imshow('img', cv2.resize(bg, (987,987)))
# cv2.waitKey()
# cv2.destroyAllWindows()

In [18]:
# Create string for g_code
g_code = "%" + "\n"
g_code += "G64 P0.001" + "\n"
g_code += "M6 T1" + "\n"
g_code += "M3 S5000" + "\n"
g_code += "G1 F" + jog_speed + "\n"
g_code += "G1X0.0000Y0.0000Z0.1000" + "\n"
g_code += "G1 F" + jog_speed + "\n"

In [19]:
# Jog to the first point in the contour
dilate_origin = tuple(contour_dilate[0][0][0])
g_code += "G1 X" + str(dilate_origin[0]/1000.00) + "Y" + str(dilate_origin[1]/1000.00) + "\n"

origin = tuple(contour[0][0])

previous_point = dilate_origin

In [20]:
# Change the contours from arrays to lists
outter = []
for point in contour_dilate[0]:
    outter.append(tuple(point[0]))
inner = []
for point in contour:
    inner.append(tuple(point[0]))

In [21]:
# Make cuts
for cut in range(cuts):
    # Plunge into the material
    g_code += "G1 F" + plunge_speed + "\n"
    current_depth -= depth_per_cut
    g_code += "G1 Z" + str(current_depth/1000.00) + "\n"

    # Cut the bigger contour
    g_code += "G1 F" + feed_speed + "\n"

    # Iterate through each point in the dilated contour
    for point in outter:
        x = point[0]/1000.00
        y = point[1]/1000.00
        g_code += "X" + str(x) + "Y" + str(y) + "\n"

      
    # Go back to the origin
    g_code += "G1 X" + str(dilate_origin[0]/1000.00) + "Y" + str(dilate_origin[1]/1000.00) + "\n"
        
    # Iterate through each point in the contour
    for point in inner:
        x = point[0]/1000.00
        y = point[1]/1000.00
        g_code += "X" + str(x) + "Y" + str(y) + "\n"
        
    # Go back to the origin
    g_code += "G1 X" + str(origin[0]/1000.00) + "Y" + str(origin[1]/1000.00) + "\n"
g_code += "G0Z.1"
g_code += "M5\n"
g_code += "M02\n"
g_code += "%"

print(g_code)

%
G64 P0.001
M6 T1
M3 S5000
G1 F15
G1X0.0000Y0.0000Z0.1000
G1 F15
G1 X0.209Y1.975
G1 F3
G1 Z-0.125
G1 F9
X0.209Y1.975
X0.209Y4.413
X0.208Y4.414
X0.078Y4.414
X0.078Y5.15
X5.975Y5.15
X5.975Y4.414
X5.842Y4.414
X5.841Y4.413
X5.841Y1.976
X5.84Y1.975
G1 X0.209Y1.975
X0.245Y2.011
X0.245Y4.449
X0.244Y4.45
X0.114Y4.45
X0.114Y5.11
X5.935Y5.11
X5.935Y4.45
X5.802Y4.45
X5.801Y4.449
X5.801Y2.012
X5.8Y2.011
X3.221Y2.011
X3.22Y2.012
X3.219Y2.011
X3.205Y2.011
X3.204Y2.012
X3.203Y2.011
X3.165Y2.011
X3.164Y2.012
X3.163Y2.011
X3.146Y2.011
X3.145Y2.012
X3.144Y2.011
X3.116Y2.011
X3.114Y2.013
X3.113Y2.012
X3.113Y2.011
X3.105Y2.011
X3.104Y2.012
X3.103Y2.011
X3.078Y2.011
X3.076Y2.013
X3.075Y2.012
X3.075Y2.011
X3.068Y2.011
X3.067Y2.012
X3.065Y2.012
X3.065Y2.013
X3.064Y2.014
X3.063Y2.013
X3.063Y2.011
X3.053Y2.011
X3.052Y2.012
X3.05Y2.012
X3.049Y2.013
X3.049Y2.146
X3.048Y2.147
X3.047Y2.146
X3.047Y2.011
X3.037Y2.011
X3.036Y2.012
X3.034Y2.012
X3.033Y2.013
X3.032Y2.012
X3.032Y2.011
X3.003Y2.011
X3.002Y2.012
X2.999Y2